<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/06_요약.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 CNN/DailyMail 데이터셋](#chapter1)
* [Chapter 2 텍스트 요약 파이프라인](#chapter2)
    * [Chapter 2-1 요약 기준 모델](#chapter2-1)
    * [Chapter 2-2 GPT-2](#chapter2-2)    
    * [Chapter 2-3 T5](#chapter2-3)    
    * [Chapter 2-4 BART](#chapter2-4)    
    * [Chapter 2-5 PEGASUS](#chapter2-5)    
* [Chapter 3 요약결과 비교](#chapter3)    
* [Chapter 4 생성된 텍스트 품질 평가하기](#chapter4)
    * [Chapter 4-1 BLEU](#chapter4-1)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. 문서의 요약은 긴 단락을 이해하고, 관련 내용을 추론하고, 원본 텍스트의 주제를 통합해 유창한 텍스트를 생성하는 능력이 필요한다.

2. 기사와 법률 계약서의 요약하는 방법은 매우 다르기 때문에 도메인에 맞는 일반화가 필요하다.

3. 사전 훈련된 트랜스포머를 사용해 몬수럴 요약하는 방법을 살펴본다.

4. 인코더-디코더 모델을 만들어 여러 사람이 주고받은 언어를 간결하게 요약해본다.

## Chapter 1 CNN/DailyMail 데이터셋 <a class="anchor" id="chapter1"></a>
1. CNN/DailyMail 데이터셋은 뉴스 기사와 해당 기사의 요약문으로 구성된 대규모 데이터셋이다.
    - 요약문은 기사에 첨부한 글머리 목록의 내용이다.
    - 요약이 본문에서 추출되지 않고 추상적이라는 중요한 특징이 있다.
    - 단순한 발췌가 아니라 새로운 문장으로 구성됐다.

2. 데이터셋은 세 가지 특성이 있다.
    - article: 뉴스 기사 본문
    - highlights: 뉴스 기사의 요약문
    - id: 각 샘플의 고유 식별자

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
print(f"특성: {dataset['train'].column_names}")

특성: ['article', 'highlights', 'id']


In [3]:
sample = dataset["train"][1]
print(f"""기사 문자 발췌, 총 길이: {len(sample['article'])}""")
print(sample["article"][:500], "...")
print(f"\n요약 (길이: {len(sample['highlights'])}):")
print(sample["highlights"])

기사 문자 발췌, 총 길이: 4051
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s ...

요약 (길이: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


3. 기사가 요약에 비해 매우 긴 경우도 있다.
    - 트랜스포모 모델의 문맥 크기가 제한적이기 때문에 긴 기사를 처리하는 데 어려움이 있다.
    - 일반적으로 긴 기사에서 중요한 정보를 추출하는 전처리 단계가 필요하다.
    - 가장 간단한 방법은 모델의 문맥의 크기에 맞게 기사를 자르는 것이다.
    - 텍스트의 끝 부분에 중요한 정보가 있다면 사라지겠지만, 이는 모델 구조의 제약으로 생기는 불가피한 손실이다.

## Chapter 2 텍스트 요약 파이프라인 <a class="anchor" id="chapter2"></a>
1. 요약 작업에 많이 사용되는 트랜스포머 모델을 몇가지 살펴본다.
    - 모델의 초대 입력 크기는 각각 다르지만 동일한 입력을 사용하기 위해 입력 텍스트를 2,000자로 제한한다.

In [4]:
sample_text = dataset["train"][1]["article"][:2000]
print(f"입력 문자 발췌, 총 길이: {len(sample_text)}")
print(sample_text, "...")
# 딕셔너리의 요약을 저장한다.
summaries = {}

입력 문자 발췌, 총 길이: 2000
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less

2. 요약에서는 관례적으로 요약 문장을 줄바꿈으로 나눈다.
    - 마침표마다 그 뒤에 줄바꿈 토큰을 추가해도 되지만, 이런 경우 "U.S."와 같은 약어가 있는 문장에서 줄바꿈이 잘못 삽입될 수 있다.
    - NLTK(Natural Language Toolkit)와 같은 라이브러리를 사용해 문장 경계를 감지하는 것이 더 정확하다.

In [11]:
#%pip install nltk
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download('punkt_tab')


string = "The U.S are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt to /home/redinblue/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/redinblue/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['The U.S are a country.', 'The U.N. is an organization.']

### Chapter 2-1 요약 기준 모델 <a class="anchor" id="chapter2-1"></a>
1. 기사를 요약하는 일반적인 기준 모델은 단순히 기사에 맨 처음 문장 세 개를 선택하는 것입니다.
    - 이는 기사의 중요한 정보가 처음에 배치되는 경향이 있기 때문입니다.
    - 이 기준 모델은 복잡한 모델과 비교할 때 성능을 평가하는 데 유용합니다.
    - NLTK 문장 토크나이져로 문장을 분리한 후 처음 세 문장을 선택합니다.

In [14]:
def three_sentence_summary(text):
    sentences = sent_tokenize(text)
    return "\n".join(sentences[:3])

summaries["baseline"] = three_sentence_summary(sample_text)
print("기준 모델 요약:")
print(summaries["baseline"])

기준 모델 요약:
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."


### Chapter 2-2 GPT-2 <a class="anchor" id="chapter2-2"></a>
1. GPT-2는 주어진 프롬프트로 텍스트를 생성할 수 있다.
    - 입력 텍스트 위에 "TL;DR"이라는 토큰을 추가해 요약을 생성하도록 유도할 수 있다.


In [15]:
# 트랜스포머늬 pipline 함수로 요약 작업을 수행한다.
from transformers import pipeline, set_seed
from nltk.tokenize import sent_tokenize

set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = f"{sample_text}\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query):]))


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


### Chapter 2-3 T5 <a class="anchor" id="chapter2-3"></a>
1. T5 모델은 "summarize:"라는 프롬프트를 사용해 요약 작업을 수행할 수 있다.
    - T5는 다양한 자연어 처리 작업을 하나의 통합된 프레임워크로 다룰 수 있는 강력한 모델이다.
    - 요약 작업에 특화된 프롬프트를 사용해 모델이 요약을 생성하도록 유도할 수 있다.
    - 요약을 포함해 여러 작업에서 비지도 학습 데이터와 지도 학습 데이터를 섞은 데이터로 훈련되었다.
    - 미세 튜닝 없이 체크포인트를 사전 훈련에 썼던 것과 동일한 프롬프트를 사용해 요약 작업을 수행할 수 있다.

2. 요약과정을 그림으로 표혀나면 아래와 같다.

     ![T5](image/05_02_t5.png) 

In [8]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"])) 

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


### Chapter 2-4 BART <a class="anchor" id="chapter2-4"></a>
1. 손상된 입력을 재구성하도록 훈련되었다.
    - BART는 인코더-디코더 아키텍처를 사용해 텍스트를 요약하는 데 효과적이다.
    - 입력 텍스트의 일부를 마스킹하거나 삭제한 후, 모델이 원본 텍스트를 재구성하도록 훈련된다.
    - 이 과정에서 모델은 중요한 정보를 추출하고 요약하는 능력을 학습한다.
    - BERT와 GPT-2의 사전 훈련 방식을 결합한다.
    - CNN/DailyMail 데이터셋에 미세 튜닝된 "facebook/bart-large-cnn" 체크포인트를 사용해 요약 작업을 수행한다.

In [9]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


### Chapter 2-5 PEGASUS <a class="anchor" id="chapter2-5ㄷ"></a>
1. 인코더-디코더 아키텍처를 사용해 텍스트 요약에 특화된 모델이다.
    - PEGASUS는 문서에서 중요한 문장을 마스킹한 후, 모델이 이를 예측하도록 훈련된다.
    - 이 방식은 요약 작업에 매우 효과적이며, 모델이 핵심 정보를 추출하는 능력을 향상시킨다.
    - CNN/DailyMail 데이터셋에 미세 튜닝된 "google/pegasus-cnn_dailymail" 체크포인트를 사용해 요약 작업을 수행한다.

2. 일반적인 언어 모델링보다 요약에 특화된 산전 훈련 목표를 찾기 위해 대규모 말뭉치에서 주변 문단의 내용을 대부분 담은 문장을 자동으로 식별했다.
    - 이러한 문장은 문서 요약에 중요한 역할을 한다.
    - PEGASUS는 이러한 문장을 마스킹하고, 모델이 이를 예측하도록 훈련된다.
    - 이 과정에서 모델은 요약에 필요한 핵심 정보를 추출하는 능력을 학습한다.
    - 그림으로 표현하면 아래와 같다.

        ![PEGASUS](image/05_03_pegasus.png)

In [10]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
# 줄바끔하는 특수 토큰이 있으므로 sent_tokenize 사용이 필요없다.
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(".</n>", ".\n").strip()

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cuda:0


## Chapter 3 요약결과 비교 <a class="anchor" id="chapter3"></a>
1. GPT-2 모델은 데디터 셋에서 전혀 훈련되지 않았다.
2. BART와 PEGASUS 모델은 CNN/DailyMail 데이터셋에 미세 튜닝되었다.

In [16]:
print("요약 결과 비교:\n")
print(dataset["train"][1]["highlights"])

for model_name in summaries:
    print(f"\n{model_name.upper()}")
    print(summaries[model_name])
    print("")


요약 결과 비교:

Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

GPT2
The mentally ill aren't being treated properly and are being housed in an environment that is not comfortable or conducive to their treatment.
A lot of the mentally ill are in jail because they are not in treatment.
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The story of the mentally ill
The s

3. 네 가지 모델 모두 정상적으로 합리적인 결과를 냈다.

4. 지표를 하나 정의하고 특정 벤치마크 데이터셋에서 모든 모델을 평가해 성능이 최고인 모델을 선택하는 것이 이상적인 방법니다.

## Chapter 4 생성된 텍스트 품질 평가하기 <a class="anchor" id="chapter4"></a>
1. 평가 지표가 나쁘면 모델의 성능 저하를 눈치 채지 못하고, 평가 지표가 비즈니스 목표에 맞지 않으면 어떤 가치도 창줄할 수 없다.

2. 생성된 텍스트를 평가하는 데 가정 널리 사용되는 두 지표는 BLEU와 ROGUE이다.

### Chapter 4-1 BLEU <a class="anchor" id="chapter4-1"></a>
1. 생성된 텍스트에서 얼마나 많은 토큰이 참조 텍스트 토큰과 완벽하게 똑같이 정렬됐는지 확인하는 대신, 단어 또는 n-그램을 체크한다.

2. 정밀도를 근간으로 하는지표이다.
    - 두텍스트를 비교할 때 참조 텍스트에 있는 단어가 생선된 텍스트에 얼마나 자주 등장하는지 카운트한다. 그 후에 생성된 텍스트 길이로 나눈다.
    - 단어를 참조 텍스트에 등장한 횟수만큼만 카운트한다.

3. 예를 들어 참조 텍스트가 "the cat is on the mat"이고 생성된 텍스트가 "the the the the the the"라고 가정하면 정밀도는 아래와 같이 계산된다.
    - pvanilla = 6 / 6 = 1
    - pmod = 2 / 6 = 0.33

4. n-그램에서 정밀도를 계산할 수도 있다.
    - P_n = ∑_C(n-gram)∈Generated Text min(Count in Generated Text, Count in Reference Text)  / ∑_C(n-gram)∈Generated Text Count in Generated Text
        - 여기서 C(n-gram)은 생성된 텍스트에 있는 모든 n-그램의 집합이다.
        - Count in Generated Text는 생성된 텍스트에서 n-그램 C의 등장 횟수이다.
    - 반복적인 생성에 보상을 주지 않도록 분자의 카둔트는 크리핑 한다.

5. 재현률를 고려하지 않기 때문에 짧지만 정밀하게 생성된 시퀸스가 긴 문장보다 유리하다.
    - 이를 해결하기 위해 생성된 텍스트 길이가 참조 텍스트 길이보다 짧을 때 패널티를 부여하는 브레브티 페널티(brevity penalty)를 도입한다.
    - BP = 1, if c > r
    - BP = exp(1 - r/c), if c ≤ r
        - 여기서 c는 생성된 텍스트의 길이, r은 참조 텍스트의 길이이다.

6. 최종 BLEU 점수는 브레브티 페널티와 n-그램 정밀도의 기하평균을 곱한 값이다.
    - BLEU = BP * exp(∑_n=1^N w_n log P_n)
        - 여기서 w_n은 n-그램 정밀도의 가중치이다. 일반적으로 모든 n에 대해 동일한 가중치를 사용한다.

7. 토큰화된 텍스트를 기대한다.
    - 일반적으로 소문자 변환, 구두점 제거, 토큰화 등의 전처리 단계를 거친다.
    - 텍스트 토큰화를 정확히 같은 방식으로 수행하지 않으면 BLEU 점수가 크게 달라질 수 있다.
    - ScareBLUE는 토큰화 단계를 내재화해 이러한 문제를 해결한다.

In [ ]:
#%pip install evaluate sacrebleu
import evaluate

bleu_metric = evaluate.load("sacrebleu")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 62.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [sacrebleu]/5 [lxml]
Note: you may need to restart the kernel to use updated packages.


8. belu_metric 개체는 MEtric 클래스의 인스턴스로 하나의 수집기 처럼 작동한다.
    - add() 메서드에 샘플 하나를 추가하거나, add_batch() 메서드에 샘플 묶음을 추가할 수 있다.
    - compute() 메서드를 호출해 최종 BLEU 점수를 계산한다.

In [19]:
import pandas as pd
import numpy as np

bleu_metric.add(predictions="the the the the the the",references=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p,2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [ ]:
bleu_metric.add(predictions="the cat is on mat",references=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p,2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])